In [8]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.utils import resample

dx = pd.read_csv('sensor_data.csv')

dx['Temp'] = pd.to_numeric(dx['Temp'], errors='coerce')
dx['Hum'] = pd.to_numeric(dx['Hum'], errors='coerce')
dx['Irr'] = pd.to_numeric(dx['Irr'], errors='coerce')

dx[['Temp', 'Irr', 'Hum']] = dx[['Temp', 'Irr', 'Hum']].fillna(dx[['Temp', 'Irr', 'Hum']].mean())
dx['Irr'] = dx['Irr'] / 120  # Approximate conversion from lux to irradiance

dx['Dew_Point'] = dx['Temp'] - ((100 - dx['Hum']) / 5)
dx['Temp_Irr_Interaction'] = dx['Temp'] * dx['Irr']
dx['Irr_Rolling_Mean'] = dx['Irr'].rolling(window=3, min_periods=1).mean()

poly = PolynomialFeatures(degree=2, include_bias=False)
poly_features = poly.fit_transform(dx[['Temp', 'Irr', 'Hum']])
poly_columns = [f'poly_{i}' for i in range(poly_features.shape[1])]
dx[poly_columns] = poly_features

synthetic_data = resample(dx, replace=True, n_samples=1000, random_state=42)
dx = pd.concat([dx, synthetic_data])

X = dx[['Temp', 'Irr', 'Hum', 'Dew_Point', 'Temp_Irr_Interaction', 'Irr_Rolling_Mean'] + poly_columns]
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

rev_model = joblib.load("solar_revenue_model.pkl")
gen_model = joblib.load("solar_generation_model.pkl")

y_pred_rev = rev_model.predict(X_test)
y_pred_rev = np.expm1(y_pred_rev)  # Reverse log transformation
y_pred_gen = gen_model.predict(X_test)

predictions_df = X_test.copy()
predictions_df['Predicted_Revenue'] = y_pred_rev
predictions_df['Predicted_Generation'] = y_pred_gen

print("\nPredictions Table (Last 10 Rows):")
print(predictions_df[['Temp', 'Irr', 'Hum', 'Predicted_Revenue', 'Predicted_Generation']].tail(10).to_string(index=False))



Predictions Table (Last 10 Rows):
 Temp      Irr  Hum  Predicted_Revenue  Predicted_Generation
 25.6 0.111083 51.7       13065.613281           3941.745850
 60.0 0.111083 51.6       11651.705078           3551.248535
 25.7 0.125000 52.8       13128.569336           3881.496582
 25.7 0.090250 52.6       13128.569336           3881.496582
 25.6 0.111083 51.7       13065.613281           3941.745850
 25.7 0.125000 52.8       13128.569336           3881.496582
 25.6 0.111083 51.7       13065.613281           3941.745850
 60.0 0.111083 51.6       11651.705078           3551.248535
 25.7 0.125000 52.8       13128.569336           3881.496582
 25.7 0.090250 52.6       13128.569336           3881.496582
